<a href="https://colab.research.google.com/github/QueeneDelmarva/Thesis/blob/main/Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Thesis Code**

## Importing Libraries

In [1]:
# Connect to Gdrive
from google.colab import drive

# Connect to Gsheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Preprocessing
import re
import nltk

import string
from google.colab import auth
from google.auth import default

## Connecting to Gdrive

In [2]:
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Connecting to GSheets

In [3]:
# Define the scope and create the credentials using the file in your Google Drive
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/skripsi-394804-a98ea9715aa4.json', scope)

# Authenticate with gspread
client = gspread.authorize(creds)

# Open a specific Google Sheet by its title or URL
sheet = client.open_by_key('1RlZ8-XwwEueuCAq4nXGwq3ZmPDE-_0gP3PsroKeNFGo').sheet1

# ## Read the data from the Gsheets
# # 1. Read individual cells
# cell_value = sheet.cell(3, 5).value
# print(cell_value)

# # 2. Read all values from the first worksheet
# data = sheet.get_all_values()

# # Print the data
# for row in data:
#     print(row)

## Preprocessing Data

In [4]:
def preprocess(word):
    # Separate words based on non-alphabetic characters
    words = re.findall(r'\b(?:[a-zA-Z0-9]+)\b', word)

    # Convert words to lowercase
    lowercase_words = [w.lower() for w in words]

    return lowercase_words

## **Radix Trie**
---
Classes:
*   Node
*   Function

### Radix Trie Node

In [10]:
# Defines a class representing a single node in the radix trie, holding information about its children and whether it marks the end of a word.
class RadixTrieNode:
    def __init__(self):
        self.children = {}
        self.is_end_of_word = False

### Radix Trie Function

In [11]:
# Defines a class that implements a radix trie data structure, allowing efficient storage and retrieval of a set of strings with common prefixes.
class RadixTrie:
    def __init__(self):
        self.root = RadixTrieNode()

    def insert(self, word):
        node = self.root
        for char in word:
            if char not in node.children:
                node.children[char] = RadixTrieNode()
            node = node.children[char]
        node.is_end_of_word = True

    def traverse(self, node=None, prefix=""):
        if node is None:
            node = self.root

        if node.is_end_of_word:
            words = prefix.split()  # Split the phrase into individual words
            for word in words:
                print("Word:", word)

        for char, child in node.children.items():
            self.traverse(child, prefix + char)

### **Main Function**

In [12]:
def main():
    # Read all values from the first worksheet
    data = sheet.get_all_values()

    # Build the Radix Trie from the preprocessed data
    trie = RadixTrie()
    for row in data:
        for word in row:
            preprocessed_words = preprocess(word)
            for preprocessed_word in preprocessed_words:
                if preprocessed_word: # Skip empty words after preprocessing
                    trie.insert(preprocessed_word)

    # Traverse and print the trie nodes and their children
    print("Nodes and Their Children:")
    trie.traverse()

if __name__ == "__main__":
    main()

Nodes and Their Children:
Word: date
Word: drop
Word: from
Word: fs0012023
Word: feb
Word: few
Word: follows
Word: fa
Word: to
Word: transport
Word: transportation
Word: t7977
Word: thanks
Word: time
Word: tumboimbella
Word: tue
Word: tba
Word: subject
Word: seletar
Word: singapore
Word: sbac
Word: content
Word: confirmed
Word: comments
Word: changi
Word: centre21
Word: crew
Word: 01
Word: 02
Word: 0840
Word: 0840lt
Word: 03
Word: 2023
Word: raymond
Word: re
Word: request
Word: road
Word: rmark
Word: koh
Word: a
Word: acam
Word: and
Word: arrange
Word: as
Word: airport
Word: aviation
Word: aerospace
Word: occ
Word: off
Word: ground
Word: wed
Word: with
Word: lt
Word: location
Word: limousine
Word: luggage
Word: noted
Word: please
Word: passenger
Word: pickup
Word: qg
Word: quwense
Word: 552
Word: eta
Word: business
Word: 1797405
Word: 1930lt
Word: vehicles
Word: jhasletha
Word: jan
Word: mpv
Word: has
Word: her
Word: hotel
Word: 31
Word: 31jan
